<a href="https://colab.research.google.com/github/eshnil2000/google-colab/blob/master/estimate_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
#Install and import all required modules

%rm -rf google-colab
!git clone https://github.com/eshnil2000/google-colab.git
%cd google-colab
!pip install web3
import web3
from web3 import Web3
#connect to blockchain instance
web3 = Web3(Web3.HTTPProvider("http://localhost:8545",request_kwargs={'timeout':60}))
print("are we connected to blockchain network?", web3.isConnected())

import config as cfg
cfg.DEBUG='TRUE'
import json
with open('contracts/Danku_demo_final.json') as json_file:  
    data = json.load(json_file)

web3.eth.defaultAccount=web3.eth.accounts[1]
myContract = web3.eth.contract(abi=data['abi'], bytecode=data['bytecode'])

tx_hash=myContract.constructor().transact()
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

# Create the contract instance with the newly-deployed address
myContractaddress = web3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=data['abi'],
)

from dutils.dataset import DemoDataset
from dutils.neural_network import NeuralNetwork
import dutils.debug as dbg
from secrets import randbelow
from matplotlib import pyplot as plt
import numpy as np

offer_account = web3.eth.accounts[1]
#solver_account = web3.eth.accounts[2]
#_hashed_data_groups = []
accuracy_criteria = 5000 # 50.00%
#total_gas_used = 0
#timeout = 180
#w_scale = 1000 # Scale up weights by 1000x
#b_scale = 1000 # Scale up biases by 1000x

scd = DemoDataset(training_percentage=0.8,partition_size=5)
scd.generate_nonce()
scd.sha_all_data_groups()

scd = DemoDataset(training_percentage=0.8,partition_size=5)
scd.generate_nonce()
scd.sha_all_data_groups()

Cloning into 'google-colab'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 330 (delta 28), reused 0 (delta 0), pack-reused 265
Receiving objects: 100% (330/330), 31.71 MiB | 32.87 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/google-colab/google-colab/google-colab/google-colab/google-colab
are we connected to blockchain network? True


In [55]:
init1_tx=myContractaddress.functions.init1(scd.hashed_data_group, accuracy_criteria,offer_account)

print("gas required", init1_tx.estimateGas())
print("gas price", web3.eth.gasPrice)
print("Total gas cost: ",init1_tx.estimateGas()* web3.eth.gasPrice)

cost=web3.fromWei(init1_tx.estimateGas()* web3.eth.gasPrice,"Ether")
print("Cost to execute function in Ether:", cost)

gas required 326624
gas price 20000000000
Total gas cost:  6532480000000000
Cost to execute function in Ether: 0.00653248


In [56]:
!pip install requests 
import requests, json
api= "https://api.cryptonator.com/api/ticker/eth-usd"
response=requests.get(api)
json_data=response.json()
eth_usd= float(json_data['ticker']['price'])
cost=float(cost)
print("Total USD cost to execute function:$",cost*eth_usd)

Total USD cost to execute function:$ 1.3665873645307136
